# Predicting Fuel Efficiency of Vehicle - Part2

## <font color='red'>Data Preparation</font>
1. Handling Categorical Attribute - one hot encoding
2. Data cleaning - Imputer
3. Attribute Addition - Adding custom transformation
4. Setting up Data Transformation Pipeline for numerical and categorical column.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedShuffleSplit

import warnings
warnings.filterwarnings('ignore')

In [2]:
cols = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 
        'Model Year', 'Origin']

df = pd.read_csv('auto-mpg.data', names=cols, na_values='?',
                comment='\t', sep=" ", skipinitialspace=True)


data = df.copy()

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data.Cylinders):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

## <font color='red'>Segregate Target and Feature variable</font>

In [3]:
data = strat_train_set.drop('MPG', axis=1)
data_labels = strat_train_set['MPG'].copy()

## <font color='red'>Preprocessing the original Column</font>

In [4]:
def preprocess_origin_cols(df):
    df['Origin'] = df['Origin'].map({1:'India', 2:'USA', 3:'Germany'})
    return df

data_tr = preprocess_origin_cols(data)
data_tr.head()

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
145,4,83.0,61.0,2003.0,19.0,74,Germany
151,4,79.0,67.0,2000.0,16.0,74,USA
388,4,156.0,92.0,2585.0,14.5,82,India
48,6,250.0,88.0,3139.0,14.5,71,India
114,4,98.0,90.0,2265.0,15.5,73,USA


## <font color='red'>One Hot Encoding the Origin Column</font>

In [5]:
data_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 145 to 362
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Cylinders     318 non-null    int64  
 1   Displacement  318 non-null    float64
 2   Horsepower    314 non-null    float64
 3   Weight        318 non-null    float64
 4   Acceleration  318 non-null    float64
 5   Model Year    318 non-null    int64  
 6   Origin        318 non-null    object 
dtypes: float64(4), int64(2), object(1)
memory usage: 19.9+ KB


In [6]:
# isolating the origin column
data_cat = data_tr[['Origin']]
data_cat.head()

,Origin
145,Germany
151,USA
388,India
48,India
114,USA


In [7]:
# onehot encoding the categorical values
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
data_cat_1hot = cat_encoder.fit_transform(data_cat)
data_cat_1hot # return a sparse matrix

<318x3 sparse matrix of type '<class 'numpy.float64'>'
	with 318 stored elements in Compressed Sparse Row format>

In [8]:
data_cat_1hot.toarray()[:5]

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [9]:
cat_encoder.categories_

[array(['Germany', 'India', 'USA'], dtype=object)]

## <font color='red'>Handling Missing values using SimpleImputer</font>

In [10]:
# segregate the numerical columns
num_data = data.iloc[:, :-1]
num_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 145 to 362
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Cylinders     318 non-null    int64  
 1   Displacement  318 non-null    float64
 2   Horsepower    314 non-null    float64
 3   Weight        318 non-null    float64
 4   Acceleration  318 non-null    float64
 5   Model Year    318 non-null    int64  
dtypes: float64(4), int64(2)
memory usage: 17.4 KB


In [11]:
# handling missing values
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
imputer.fit(num_data)

SimpleImputer(strategy='median')

In [12]:
# median of all the columns from imputer
imputer.statistics_

array([   4. ,  146. ,   92. , 2844. ,   15.5,   76. ])

In [13]:
# median from pandas dataframe - same
data.median().values

array([   4. ,  146. ,   92. , 2844. ,   15.5,   76. ])

In [14]:
# imputing the missing values by transforming the dataframe
x = imputer.transform(num_data)
x

array([[   4. ,   83. ,   61. , 2003. ,   19. ,   74. ],
       [   4. ,   79. ,   67. , 2000. ,   16. ,   74. ],
       [   4. ,  156. ,   92. , 2585. ,   14.5,   82. ],
       ...,
       [   4. ,  135. ,   84. , 2295. ,   11.6,   82. ],
       [   4. ,  113. ,   95. , 2372. ,   15. ,   70. ],
       [   6. ,  146. ,  120. , 2930. ,   13.8,   81. ]])

In [15]:
# converting the 2d array back to a dataframe

data_tr = pd.DataFrame(x, columns=num_data.columns, index=num_data.index)
data_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 145 to 362
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Cylinders     318 non-null    float64
 1   Displacement  318 non-null    float64
 2   Horsepower    318 non-null    float64
 3   Weight        318 non-null    float64
 4   Acceleration  318 non-null    float64
 5   Model Year    318 non-null    float64
dtypes: float64(6)
memory usage: 17.4 KB


## <font color='red'>Adding Attributes using BaseEstimator and Transformer</font>

In [16]:
num_data.head()

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year
145,4,83.0,61.0,2003.0,19.0,74
151,4,79.0,67.0,2000.0,16.0,74
388,4,156.0,92.0,2585.0,14.5,82
48,6,250.0,88.0,3139.0,14.5,71
114,4,98.0,90.0,2265.0,15.5,73


In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

acc_ix, hpower_ix, cyl_ix = 3, 5, 1

class CustomAttrAdder(BaseEstimator, TransformerMixin):
    def __init__(self, acc_on_power=True):
        self.acc_on_power = acc_on_power # no *args or **kwargs
    def fit(self, x, y=None):
        return self # nothing else to do
    def transform(self, x):
        acc_on_cyl = x[:, acc_ix]/x[:, cyl_ix]
        if self.acc_on_power:
            acc_on_power = x[:, acc_ix] / x[:, hpower_ix]
            return np.c_[x, acc_on_power, acc_on_cyl] # np.c_ will concate numpy array
        return np.c_[x, acc_on_cyl]
    
attr_adder = CustomAttrAdder(acc_on_power=True)
data_tr_extra_attrs = attr_adder.transform(data_tr.values)
data_tr_extra_attrs[0]

array([   4.        ,   83.        ,   61.        , 2003.        ,
         19.        ,   74.        ,   27.06756757,   24.13253012])

previously we had 6 columns but after adding acceleration_on_cylinder & acceleration_on_power our data have 8 rows

## <font color='red'>Creating a Pipeline of tasks</font>

In [18]:
# Using pipline class
from sklearn.pipeline import Pipeline

# Using StandardScalar to scale all the numerical attributes
from sklearn.preprocessing import StandardScaler

numerics = ['float64', 'int64']

num_data = data_tr.select_dtypes(include=numerics)

# pipeline for numerical attributes
# imputing -> adding attributes -> scale them

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attrs_adder', CustomAttrAdder()), # this is a custom class we have created
    ('std_scaler', StandardScaler())
])

num_data_tr = num_pipeline.fit_transform(num_data)
num_data_tr[0]

array([-0.85657842, -1.07804475, -1.15192977, -1.17220298,  1.21586943,
       -0.54436373, -1.04758895,  1.33350893])

## <font color='red'>Transforming Numerical and Categorical Attributes</font>

In [19]:
# Transform different columns or subsets using columnTransformer
from sklearn.compose import ColumnTransformer

num_attrs = list(num_data)
cat_attr = ['Origin']

# complete pipleline to transform
# both numerical and cat attributes
full_pipeline =ColumnTransformer([
    ('num', num_pipeline, num_attrs),
    ('cat', OneHotEncoder(), cat_attr)
])

prepared_data = full_pipeline.fit_transform(data)
prepared_data

array([[-0.85657842, -1.07804475, -1.15192977, ...,  1.        ,
         0.        ,  0.        ],
       [-0.85657842, -1.1174582 , -0.9900351 , ...,  0.        ,
         0.        ,  1.        ],
       [-0.85657842, -0.3587492 , -0.31547399, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.85657842, -0.56566984, -0.53133355, ...,  0.        ,
         1.        ,  0.        ],
       [-0.85657842, -0.78244384, -0.23452666, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.32260746, -0.45728283,  0.44003446, ...,  1.        ,
         0.        ,  0.        ]])